# Decision Theory Project - Chess
*By Jelle Huibregtse and Aron Hemmes*

Below is a Chess environment build from scratch with an Agent based on reward

## 1. Setup
- Loading in some libraries
- Configuring layout and styling

In [1]:
# Libraries
import random
from enum import Enum
from typing import List
from IPython.display import display, clear_output
from ipywidgets import Layout, Button, Box

# Layout
field_layout = Layout(width = '50px', height = '50px', margin = '0', padding = '0')
close_layout = Layout(width = '50px', margin = '0', padding = '0')
column_layout = Layout(flex_flow = 'column')

In [2]:
%%HTML
<style>
.button {
    outline:none !important;
    box-shadow:none !important;
    cursor: default;
}

.row .button, .promotion_window .button:nth-child(-n+4) {
    font-size:350%;
}

.white {
    color: white !important;
}

.selectable {
    cursor: pointer;
}

.board {
    padding: 5px 0;
    margin: 0 !important;
}

.output_subarea {
    padding: 0 !important;
}

.promotion_window {
    border-radius: 0 0 3px 3px;
    box-shadow: 3px 3px 10px rgb(0 0 0 / 45%);
    width: 50px;
    position: absolute;
    z-index: 1000;
    margin-top: 5px;
    background-color: var(--jp-layout-color3);
}

.promotion_window .button {
    background-color: transparent;
}

.circle_filled::before {
    content: '';
    width: 40%;
    height: 40%;
    border-radius: 50%;
    position: absolute;
    background: rgb(51, 82, 107);
    top: 30%;
    left: 30%;
}

.circle::before {
    content: '';
    width: 100%;
    height: 100%;
    border: rgb(51, 82, 107) 5px solid;
    border-radius: 50%;
    position: absolute;
    top: 0;
    left: 0;
}

.selected::before {
    content: '';
    width: 100%;
    height: 100%;
    border: rgb(162, 185, 204) solid 5px;
    position: absolute;
    top: 0;
    left: 0;
}
</style>

## 2. Definition of the Environment

the code below defines all characteristics of a Chess Environment:

In [3]:
class Color(Enum):
    WHITE = 1
    BLACK = 2

class PieceType(Enum):
    KING = '♚'
    QUEEN = '♛'
    ROOK = '♜'
    BISHOP = '♝'
    KNIGHT = '♞'
    PAWN = '♟︎'

class Action(Enum):
    Move = 1
    Take = 2
    Castle = 3
    EnPassant = 4

class Move:
    def __init__(self, action : Action, x_start : int, y_start : int, x_end : int, y_end : int, turn: int, promoting : PieceType = None):
        self.action = action
        self.x_start = x_start
        self.y_start = y_start
        self.x = x_end
        self.y = y_end
        self.turn = turn
        self.promoting = promoting

class Piece:
    def __init__(self, x : int, y : int, type : PieceType, color : Color):
        self.x = x
        self.y = y
        self.color = color
        self.type = type
        self.history = []
    
    def execute_move(self, move : Move):
        self.history.append(move)
        self.x = move.x
        self.y = move.y
        self.type = move.promoting if not move.promoting == None else self.type
    
    def set_pos(self, x, y):
        self.x = x
        self.y = y
    
    def take(self):
        self.x = None
        self.y = None
    
    def get_legal_moves(self, board) -> List[Move]:
        return MoveGenerator().generate_legal_moves(board, self)

class Board:
    def __init__(self, engine : any, player_color : Color = Color.WHITE, pieces : List[Piece] = None):
        self.engine = engine
        
        # Setting player color
        self.player_color = player_color
        # Adding pieces to board
        self.pieces = pieces if not pieces == None else self.generate_default()
        # Board variables
        self.promotion_window = None
        self.turn = 0
        self.fields = []
        
        # Player variables
        self.player_selected_moves = []
        self.target_move = None
    
    # -----------------------------------
    # Board Util
    # -----------------------------------
    def restart(self):
        self.pieces = self.generate_default()
        self.player_color = Color.BLACK if self.player_color == Color.WHITE else Color.BLACK

    def clear_board(self):
        self.pieces = []

    def get_piece(self, x, y) -> Piece:
        pieces = [piece for piece in self.pieces if piece.x == x and piece.y == y]
        return pieces[0] if len(pieces) > 0 else None

    def execute_move(self, move : Move):
        piece = self.get_piece(move.x_start, move.y_start)
        target_piece = None
        if move.action == Action.Take:
            target_piece = self.get_piece(move.x, move.y)
        elif move.action == Action.EnPassant:
            target_piece = self.get_piece(move.x, piece.y)
        elif move.action == Action.Castle:
            target_piece = self.get_piece(0 if move.x < move.x_start else 7, move.y)
        
        if move.action == Action.Castle:
            target_piece.set_pos(move.x + (1 if move.x < move.x_start else -1), target_piece.y)
        
        elif not target_piece == None:
            target_piece.take()
        
        piece.execute_move(move)
        
        # Re-render board
        for i in range(len(self.fields)):
            self.render_piece(i)
    
    # Handles clicking on fields
    def field_click(self, e):
        # Clicked location variables
        clicked_field = int(e.tooltip)
        clicked_x = clicked_field % 8
        clicked_y = int(clicked_field / 8)
        
        # Hide promotion window
        self.close_promotion_window('')
        
        # Hide selection and move indicators
        for field in self.fields:
            field.remove_class('circle_filled')
            field.remove_class('circle')
            field.remove_class('selected')
        
        # Getting move on clicked location
        move = [move for move in self.player_selected_moves if move.x == clicked_x and move.y == clicked_y]
        move = move[0] if any(move) else None
        
        # Open promotion window
        promotion = False
        piece = self.get_piece(move.x_start, move.y_start) if not move == None else None
        if not piece == None and piece.type == PieceType.PAWN and ((move.y == 0 and self.player_color == Color.WHITE) or (move.y == 7 and self.player_color == Color.BLACK)):
            self.target_move = move
            self.open_promotion_window(clicked_field if self.player_color == Color.WHITE else 63 - clicked_field)
            promotion = True
        
        # Execute move
        if not promotion and not move == None:
            self.execute_move(move)
        
        # Reset player selected moves
        self.player_selected_moves = []
        
        # Add moves for clicked piece
        if not promotion and move == None:
            clicked_piece = self.get_piece(clicked_x, clicked_y)
            if not clicked_piece == None and clicked_piece.color == self.player_color and (len(self.player_selected_moves) == 0 or not (self.player_selected_moves[0].x_start == clicked_x and self.player_selected_moves[0].y_start == clicked_y)):
                # Add clicked piece's moves to player_selected_moves
                self.player_selected_moves = clicked_piece.get_legal_moves(self)

                # Show selected outline
                field = clicked_piece.x + clicked_piece.y * 8 if self.player_color == Color.WHITE else 63 - clicked_piece.x + clicked_piece.y * 8
                self.fields[field].add_class('selected')
                
                # Show move indicators
                for move in self.player_selected_moves:
                    f = move.x + move.y * 8
                    if self.get_piece(move.x, move.y) == None:
                        self.fields[f if self.player_color == Color.WHITE else 63 - f].add_class('circle_filled')
                    else:
                        self.fields[f if self.player_color == Color.WHITE else 63 - f].add_class('circle')

    # Clicked on promote button in promotion window
    def promote_piece(self, e):
        selected_piece = self.get_piece(self.target_move.x_start, self.target_move.y_start) if not self.target_move == None else None
        target_piece = self.get_piece(self.target_move.x, self.target_move.y) if not self.target_move == None else None
        if not selected_piece == None:
            if not target_piece == None:
                target_piece.take()
            self.target_move.promoting = PieceType(e.description)
            selected_piece.execute_move(self.target_move)
            self.render_piece(self.target_move.x_start + self.target_move.y_start * 8)
            self.render_piece(self.target_move.x + self.target_move.y * 8)

            self.close_promotion_window('')

    # Clicked on close button in promotion window
    def close_promotion_window(self, _):
        self.promotion_window.add_class('hidden')

    def open_promotion_window(self, field: int):
        self.promotion_window.layout.margin = '5px 0 0 ' + str((field) * 50) + 'px'
        self.promotion_window.remove_class('hidden')

    # Generates the default 8 * 8 chess board pieces
    def generate_default(self) -> List[Piece]:
        side = ['♜', '♞', '♝', '♛', '♚',  '♝', '♞', '♜'] + ['♟︎' for _ in range(8)]
        p = side + [' ' for _ in range(32)] + side[::-1]
        pieces = []
        for i in range(len(p)):
            piece = p[i]
            if piece != ' ':
                x = i % 8
                y = int(i / 8)
                pieces.append(Piece(x, y, PieceType(piece), Color.BLACK if i < 32 else Color.WHITE))
                
        return pieces
    
    # -----------------------------------
    # Rendering Util
    # -----------------------------------
    def render_piece(self, field, piece = None):
        if piece == None:
            piece = self.get_piece(field % 8, int(field / 8))
        
        f = field if self.player_color == Color.WHITE else 63 - field
        self.fields[f].description = piece.type.value if not piece == None else ' '
        if not piece == None and piece.color == Color.WHITE:
            self.fields[f].add_class('white')
        else:
            self.fields[f].remove_class('white')
        
        if not piece == None and self.player_color == piece.color:
            self.fields[f].add_class('selectable')
        else:
            self.fields[f].remove_class('selectable')
    
    def generate_field(self, x, y) -> Button:
        field = x + y * 8
        btn = Button(layout = field_layout, tooltip = str(field))
        # Setting grid color
        btn.style = {'button_color': '#7495b1' if (x + y) % 2 == 0 else '#477397'}
        btn.add_class('button')
        btn.on_click(self.field_click)
        self.fields.append(btn)
        self.render_piece(field)
        
        return self.fields[-1]
    
    # A promotion window for promoting pawns
    def generate_promotion_window(self):
        buttons = []
        for x in range(4):
            piece = '♞' if x == 1 else '♜' if x == 2 else '♝' if x == 3 else '♛'
            btn = Button(layout = field_layout, description = piece)
            btn.add_class('button')
            btn.add_class('selectable')
            if self.player_color == Color.WHITE:
                btn.add_class('white')
            btn.on_click(self.promote_piece)
            buttons.append(btn)
        close = Button(layout = close_layout, description = '🗙')
        close.add_class('button')
        close.add_class('selectable')
        if self.player_color == Color.WHITE:
            close.add_class('white')
        close.on_click(self.close_promotion_window)
        buttons.append(close)
        promotion = Box(children = buttons, layout = column_layout)
        promotion.add_class('promotion_window')
        promotion.add_class('hidden')
        
        return promotion
    
    # Rendering a custom chess board
    def render(self):
        # Clear output
        clear_output(wait=True)
        
        # Add promotion window
        promotion = self.generate_promotion_window()
        self.promotion_window = promotion
        
        # Add board
        rows = []
        spaces = (list(range(8)) if self.player_color == Color.WHITE else list(range(8))[::-1])
        for y in spaces:
            row = Box([self.generate_field(x, y) for x in spaces])
            row.add_class('row')
            rows.append(row)
        
        board = Box(children = rows, layout = column_layout)
        board.add_class('board')
        
        # Display elements
        display(promotion)
        display(board)

## 3. Valid move generator

In [4]:
class MoveGenerator:
    # A dynamic list of legal moves for a piece
    def generate_legal_moves(self, board : Board, piece: Piece) -> List[Move]:
        moves = self.generate_pseudo_legal_moves(board, piece)
        
        if piece.type == PieceType.KING:
            # TODO: LEGAL MOVES FOR KING
            print("moves for king are pseudo legal!")
        
        return moves
    
    # Pseudo-legal moves might leave or put the king in check, but are otherwise valid
    def generate_pseudo_legal_moves(self, board : Board, piece: Piece) -> List[Move]:
        moves = []
        
        if piece.type == PieceType.PAWN:
            for x, y in [[piece.x + x, piece.y + (y if not piece.color == Color.WHITE else -y)] for x, y in ([[0, 1], [1, 1], [-1, 1]] + ([[0, 2]] if len(piece.history) == 0 else []))]:
                if 7 >= x >= 0 and 7 >= y >= 0:
                    # If no pieces in between
                    if not abs(piece.y - y) == 2 or (abs(piece.y - y) == 2 and board.get_piece(x, int((piece.y + y) / 2)) == None):
                        target_piece = board.get_piece(x, y)
                        if x == piece.x and target_piece == None:
                            moves.append(Move(Action.Move, piece.x, piece.y, x, y, board.turn))
                        if not x == piece.x and not target_piece == None and not target_piece.color == piece.color:
                            moves.append(Move(Action.Take, piece.x, piece.y, x, y, board.turn))
            
            # En Passant
            if (piece.y == 3 and piece.color == Color.WHITE) or (piece.y == 4 and piece.color == Color.BLACK):
                for x, y in [[piece.x + x, piece.y + (y if not piece.color == Color.WHITE else -y)] for x, y in [[1, 1], [-1, 1]]]:
                    target_piece = board.get_piece(x, piece.y)
                    if not target_piece == None and target_piece.type == PieceType.PAWN:
                        move = target_piece.history[-1] if len(target_piece.history) > 0 else None
                        if not move == None and move.turn == board.turn - 1 and abs(move.y_start - move.y) == 2:
                            moves.append(Move(Action.EnPassant, piece.x, piece.y, x, y, board.turn))
        
        elif piece.type == PieceType.KNIGHT:
            for m in [1, -1]:
                for x, y in [[piece.x + x * m, piece.y + y * m] for x, y in [[1, 2], [-1, 2], [2, 1], [-2, 1]]]:
                    if 7 >= x >= 0 and 7 >= y >= 0:
                        target_piece = board.get_piece(x, y)
                        if target_piece == None or (not target_piece == None and not target_piece.color == piece.color):
                            moves.append(Move(Action.Take, piece.x, piece.y, x, y, board.turn))
        
        elif piece.type == PieceType.KING:
            for m in [1, -1]:
                for x, y in [[piece.x + x * m, piece.y + y * m] for x, y in [[0, 1], [1, 1], [1, 0], [1, -1]]]:
                    if 7 >= x >= 0 and 7 >= y >= 0:
                        target_piece = board.get_piece(x, y)
                        if target_piece == None or (not target_piece == None and not target_piece.color == piece.color):
                            moves.append(Move(Action.Take, piece.x, piece.y, x, y, board.turn))
            
            # Castling
            if len(piece.history) == 0:
                for x, y in [[0, piece.y], [7, piece.y]]:
                    target_piece = board.get_piece(x, y)
                    
                    # If no pieces in between
                    if not any([p for p in [board.get_piece(x, y) for x in range(min(x, piece.x) + 1, max(x, piece.x))] if not p == None]):
                        if not target_piece == None and target_piece.type == PieceType.ROOK and len(target_piece.history) == 0:
                            a, b = [piece.x - 2 if x < piece.x else piece.x + 2, piece.y]
                            if 7 >= a >= 0 and 7 >= b >= 0:
                                moves.append(Move(Action.Castle, piece.x, piece.y, a, b, board.turn))
        
        elif piece.type == PieceType.ROOK or piece.type == PieceType.QUEEN:
            for x, y in [[0, 1], [0, -1], [1, 0], [-1, 0]]:
                for x, y in [[piece.x + x * m, piece.y + y * m] for m in range(1, 8)]:
                    if 7 >= x >= 0 and 7 >= y >= 0:
                        target_piece = board.get_piece(x, y)
                        if target_piece == None:
                            moves.append(Move(Action.Take, piece.x, piece.y, x, y, board.turn))
                        elif not target_piece == None and not target_piece.color == piece.color:
                            moves.append(Move(Action.Take, piece.x, piece.y, x, y, board.turn))
                            break
                        else:
                            break
        
        if piece.type == PieceType.BISHOP or piece.type == PieceType.QUEEN:
            for x, y in [[1, 1], [-1, -1], [1, -1], [-1, 1]]:
                for x, y in [[piece.x + x * m, piece.y + y * m] for m in range(1, 8)]:
                    if 7 >= x >= 0 and 7 >= y >= 0:
                        target_piece = board.get_piece(x, y)
                        if target_piece == None:
                            moves.append(Move(Action.Take, piece.x, piece.y, x, y, board.turn))
                        elif not target_piece == None and not target_piece.color == piece.color:
                            moves.append(Move(Action.Take, piece.x, piece.y, x, y, board.turn))
                            break
                        else:
                            break
        
        return moves

## 4. Random

In [5]:
board = Board(None)
board.render()

Box(children=(Button(description='♛', layout=Layout(height='50px', margin='0', padding='0', width='50px'), sty…

Box(children=(Box(children=(Button(description='♜', layout=Layout(height='50px', margin='0', padding='0', widt…